# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from math import sin, cos, sqrt, atan2, radians
# Module to reverse geocode hotel data. Install with "pip install geopy"
import reverse_geocoder as rg

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame

cities_pd = pd.read_csv("output_data/output.csv")
cities_pd.head()

,City,ID,Lat,Long,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,yellowknife,6185377,62.4560,-114.3525,67.96,43,5,12.17,CA,Sat Jun 26 21:11:20 2021
1,atuona,4020109,-9.8000,-139.0333,78.94,66,6,21.25,PF,Sat Jun 26 21:11:20 2021
2,san quintin,3984997,30.4833,-115.9500,64.20,84,0,11.56,MX,Sat Jun 26 21:11:20 2021
3,hasaki,2112802,35.7333,140.8333,79.18,81,100,8.34,JP,Sat Jun 26 21:11:21 2021
4,port alfred,964432,-33.5906,26.8910,53.40,79,61,6.78,ZA,Sat Jun 26 21:11:21 2021


### Humidity Heatmap

In [3]:
#Configure gmaps, load API key

gmaps.configure(g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.

locations = cities_pd[["Lat", "Long"]].astype(float)
city_humidity = cities_pd["Humidity"].astype(float) 

In [5]:
# Identify highest humidity level for weight

heat_max = (cities_pd["Humidity"].max()).astype(float)
print(heat_max)

100.0


In [6]:
# Print heat map

center_coord = (40.52,34.34)
fig = gmaps.figure(center=center_coord,zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=heat_max,
                                 point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Preview DataFrame to verify structure

cities_df = pd.DataFrame(cities_pd)
cities_df

,City,ID,Lat,Long,Max Temp,Humidity,Cloudiness,Wind,Country,Date
0,yellowknife,6185377,62.4560,-114.3525,67.96,43,5,12.17,CA,Sat Jun 26 21:11:20 2021
1,atuona,4020109,-9.8000,-139.0333,78.94,66,6,21.25,PF,Sat Jun 26 21:11:20 2021
2,san quintin,3984997,30.4833,-115.9500,64.20,84,0,11.56,MX,Sat Jun 26 21:11:20 2021
3,hasaki,2112802,35.7333,140.8333,79.18,81,100,8.34,JP,Sat Jun 26 21:11:21 2021
4,port alfred,964432,-33.5906,26.8910,53.40,79,61,6.78,ZA,Sat Jun 26 21:11:21 2021
...,...,...,...,...,...,...,...,...,...,...
556,lappeenranta,648900,61.0587,28.1887,64.45,100,90,3.44,FI,Sat Jun 26 21:10:30 2021
557,mogadishu,53654,2.0371,45.3438,77.77,83,81,14.85,SO,Sat Jun 26 21:10:09 2021
558,arzgir,580716,45.3728,44.2206,76.35,64,97,14.99,RU,Sat Jun 26 21:14:36 2021
559,nuevo ideal,3994607,24.8875,-105.0728,72.00,30,11,6.73,MX,Sat Jun 26 21:14:37 2021


In [8]:
# Ideal weather conditions: 75-85F, Wind Speed 5-25mph, humidity 10-60%. Confirm reasonable results count

cities_ideal = cities_df[(cities_df['Max Temp'] < 82) & 
            (cities_df['Wind'] >= 5) & (cities_df['Wind'] <= 25) & 
            (cities_df['Humidity'] >= 15) & (cities_df['Humidity'] <= 55)]

cities_ideal_df = pd.DataFrame(cities_ideal)
cities_ideal_df.dropna(inplace=True)
print(len(cities_ideal_df))

46


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Data cleaning

cities_ideal_df=cities_ideal_df.drop(columns=["Date"])
cities_ideal_df["Hotel Name"] = ""

In [10]:
# Create function to verify search result distance. This is to counter Google's default of using home IP address lat/long if no result is found.

def geospatial(search_lng, search_lat, result_lon, result_lat):
    R = 6373.0

    lat1 = radians(search_lat)
    lon1 = radians(search_lng)
    lat2 = radians(result_lat)
    lon2 = radians(result_lon)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return(distance)

In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&fields=formatted_address,name,geometry&locationbias=circle:5000@"

hotel_names = []
hotel_lat = []
hotel_lng = []

# use iterrows to iterate through pandas dataframe
for index, row in cities_ideal_df.iterrows():
    search_city = row["City"]
    search_lat = row["Lat"]
    search_lng = row["Long"]

    # assemble url and make API request
    response = requests.get(f"{base_url}{search_lat},{search_lng}&key={g_key}").json()
    candidates = response['candidates'][0]

    result_lat = candidates['geometry']['location']['lat']
    result_lon = candidates['geometry']['location']['lng']
    hotel_name = candidates['name']

   # Verify results are within 5000m, otherwise skip row.
    distance = geospatial(search_lng, search_lat, result_lon, result_lat)
    if distance > 5:
       continue

    hotel_names.append(hotel_name)
    hotel_lat.append(result_lat)
    hotel_lng.append(result_lon)

In [12]:
# Create DataFrame with just hotel information

hotel_info_df = pd.DataFrame(list(zip(hotel_names,hotel_lat,hotel_lng)),columns=["Name","Lat","Lng"])

In [13]:
# Retrieve hotel's city and country using reverse geocode search. Note that reverse_geocoder must be installed for this to work.

city_list = []
country_list = []

for row in hotel_info_df.iterrows():
    coordinates = (row[1]["Lat"],row[1]["Lng"])
    result = rg.search(coordinates)
    result_df = pd.DataFrame(result)
    city_list.append(result_df['name'])
    country_list.append(result_df['cc'])

Loading formatted geocoded file...


In [14]:
# Finalize Hotel DataFrame by adding City and Country

city_list = list(map(lambda x:x[0],city_list)) 
country_list = list(map(lambda x:x[0],country_list)) 

hotel_df = pd.DataFrame(list(zip(hotel_names,city_list,country_list,hotel_lat,hotel_lng)),columns=["Hotel Name","City","Country","Lat","Lng"])

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
marker_locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))